In [ ]:
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not set")

if not wandb_api_key:
    raise ValueError("WANDB_API_KEY not set")

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)

from peft import (
    LoraConfig,
    get_peft_model
)

import wandb
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login

login(token=huggingface_api_key)

In [ ]:
wandb.login(key=wandb_api_key)

run = wandb.init(
    project="repurposed-llm-phishing-classifier",
    job_type="train",
    anonymous="allow"
)

In [ ]:
base_model_path = "../models/llama_models/llama-3.2-3B"
new_model_path = "../models/tuned_models/llama-3.2-3B-phishing-classifier"
train_dataset_path = "../processed_data/train.csv"
test_dataset_path = "../processed_data/test.csv"

In [ ]:
train_df = pd.read_csv(train_dataset_path)

train_df.head()

In [ ]:
model_config = {
    "torch_dtype": torch.bfloat16,
    "attn_implementation": "flash_attention_2",
    "device_map": "auto"
}

In [ ]:
# WARNING: flash_attention_2 required pip install flash-attn which needs C++ builds tools
# It also takes absolutely forever to compile because it's compiling CUDA kernels

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map=model_config["device_map"],
    torch_dtype=model_config["torch_dtype"],
    attn_implementation=model_config["attn_implementation"]
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

In [ ]:
tokenizer.chat_template = """
{% for message in messages %}
<|begin_of_text|><|start_header_id|>{{ message['role'] }}<|end_header_id|>
{{ message['content'] }}
<|eot_id|>{% endfor %}
"""

In [ ]:
test_messages = [
    {"role": "system", "content": "You are a classification system..."},
    {"role": "user", "content": "Message for review: test"},
    {"role": "assistant", "content": "true"}
]

print("Test output:", tokenizer.apply_chat_template(test_messages, tokenize=False))

In [ ]:
from functools import partial

train_dataset = Dataset.from_pandas(pd.read_csv(train_dataset_path))

def format_chat_template_batch(examples, tokenizer):  # Add tokenizer as parameter
    formatted_texts = []

    for system, user, assistant in zip(
        examples["system"],
        examples["user"],
        examples["assistant"]
    ):
        row_json = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "assistant", "content": str(assistant)}
        ]
        formatted_texts.append(tokenizer.apply_chat_template(row_json, tokenize=False))

    examples["text"] = formatted_texts
    return examples


format_with_tokenizer = partial(format_chat_template_batch, tokenizer=tokenizer)

train_dataset = train_dataset.map(
    format_with_tokenizer,
    batched=True,
    batch_size=100
)

In [ ]:
test_dataset = Dataset.from_pandas(pd.read_csv(test_dataset_path))

test_dataset = test_dataset.map(
    format_with_tokenizer,
    batched=True,
    batch_size=100
)

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False
)

model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir=new_model_path,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=1,
    warmup_steps=100,
    learning_rate=5e-5,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb",
    neftune_noise_alpha=0.1,
    run_name="actual_run_1",
    logging_first_step=True,
    logging_dir="../logs",
    logging_strategy="steps",
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    args=training_arguments
)

In [ ]:
trainer.train()